In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv(r"c:\Users\USER\Desktop\MyDatasets\heart.csv")
df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


# Preparing the data.
1. First we replace categorical data with numeric values

In [3]:
df.Sex.replace(['F', 'M'], [0, 1], inplace=True)
df.ChestPainType.replace(['ATA', 'NAP', 'ASY', 'TA'], [0, 1, 2, 3], inplace=True)
df.RestingECG.replace(['Normal', 'ST', 'LVH'], [0, 1, 2], inplace=True)
df.ExerciseAngina.replace(['N', 'Y'], [0, 1], inplace=True)
df.ST_Slope.replace(['Up', 'Flat', 'Down'], [0, 1, 2], inplace=True)
df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,0,140,289,0,0,172,0,0.0,0,0
1,49,0,1,160,180,0,0,156,0,1.0,1,1
2,37,1,0,130,283,0,1,98,0,0.0,0,0
3,48,0,2,138,214,0,0,108,1,1.5,1,1
4,54,1,1,150,195,0,0,122,0,0.0,0,0
5,39,1,1,120,339,0,0,170,0,0.0,0,0
6,45,0,0,130,237,0,0,170,0,0.0,0,0
7,54,1,0,110,208,0,0,142,0,0.0,0,0
8,37,1,2,140,207,0,0,130,1,1.5,1,1
9,48,0,0,120,284,0,0,120,0,0.0,0,0


2. We convert the dataframe to a numpy array **and** split the dataset into train and test segments 

In [4]:
data = df.to_numpy()

X = data[:, :11]
Y = data[:, 11]

X_train, X_test, Y_train, Y_test = tts(X, Y, test_size=0.2, random_state=1, shuffle=True)

3. We scale our data, as most machine learning models do better when the input data is scaled or normalised.

In [5]:
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

I'll be using the sklearn library for the most part of the classification.

# Sci-kit learn 
### Decision Tree Classifier

In [6]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=1)
clf.fit(X_train_scaled, Y_train)

accuracy = round(((clf.score(X_test_scaled, Y_test)) * 100 ))
print(f'Accuracy of model: {accuracy}%')

Accuracy of model: 78%


### Let's try something cheeky with the K-Fold cross_validation
Perhaps we could improve the accuracy of the DecisionTreeClassifier

In [7]:
from sklearn.model_selection import cross_val_score
clf = DecisionTreeClassifier(random_state=1)
cv_scores = cross_val_score(clf, X, Y, cv=26)
accuracy = round((cv_scores.mean()) * 100)
print(f'Accuracy of model: {accuracy}%')

Accuracy of model: 79%


A little improvement. 
Generally, basic decision tree isn't so much of an efficient model choice.

### RandomForestClassifier

In [8]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=35, random_state=1)
clf.fit(X_train_scaled, Y_train)
accuracy = round((clf.score(X_test_scaled, Y_test) * 100))
print(f'Accuracy of model: {accuracy}%')

Accuracy of model: 90%


***Wow***!!! what a jump. 11% improvement on accuracy.

Now we could go ahead and use the K-Fold cross validation; there might be room for  more improvement.

### SVM

In [9]:
from sklearn import svm
svc = svm.SVC(kernel='rbf', C=1.0)
svc.fit(X_train_scaled, Y_train)
accuracy = round((svc.score(X_test_scaled, Y_test))*100)
print(f'Accuracy of model: {accuracy}%')

Accuracy of model: 91%


The SVM model has upped accuracy by 1%, impressive.

### KNN (K-Nearest Neighbours)

In [10]:
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=10)
clf.fit(X_train_scaled, Y_train)
accuracy = round((svc.score(X_test_scaled, Y_test)) * 100)
print(f'Accuracy of model: {accuracy}%')

Accuracy of model: 91%


When working with **KNNs**, choosing the K value is a bit tricky, so it's best to create a loop that train the model over different values of K. <br>
Then we can go ahead to  select the K value with the highest accuracy.

In [11]:
for i in range(1, 51):
    clf = neighbors.KNeighborsClassifier(n_neighbors=i)
    clf.fit(X_train_scaled, Y_train)
    accuracy = round((svc.score(X_test_scaled, Y_test))*100)
    print(f'Accuracy of model using K value of {i}: {accuracy}%')

Accuracy of model using K value of 1: 91%
Accuracy of model using K value of 2: 91%
Accuracy of model using K value of 3: 91%
Accuracy of model using K value of 4: 91%
Accuracy of model using K value of 5: 91%
Accuracy of model using K value of 6: 91%
Accuracy of model using K value of 7: 91%
Accuracy of model using K value of 8: 91%
Accuracy of model using K value of 9: 91%
Accuracy of model using K value of 10: 91%
Accuracy of model using K value of 11: 91%
Accuracy of model using K value of 12: 91%
Accuracy of model using K value of 13: 91%
Accuracy of model using K value of 14: 91%
Accuracy of model using K value of 15: 91%
Accuracy of model using K value of 16: 91%
Accuracy of model using K value of 17: 91%
Accuracy of model using K value of 18: 91%
Accuracy of model using K value of 19: 91%
Accuracy of model using K value of 20: 91%
Accuracy of model using K value of 21: 91%
Accuracy of model using K value of 22: 91%
Accuracy of model using K value of 23: 91%
Accuracy of model us

Apparently, K values doesn't affect the model performance. But this is doesn't happen too often.

### Naive Bayes
It's important to note that doesn't take negative feature values, as regarding our earlier scaled features. <br>
So we'll have to use the MinMaxScaler instead of the StandardScaler

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

# scale the input features
scaler = MinMaxScaler()
X_train_minmax = scaler.fit_transform(X_train)
X_test_minmax = scaler.transform(X_test)

# define the classifier
clf = MultinomialNB()
clf.fit(X_train_minmax, Y_train)
accuracy = round((clf.score(X_test_minmax, Y_test)) * 100)
print(f'Accuracy of the model: {accuracy}%')

Accuracy of the model: 89%


Naive Bayes is 2 points back from the accuracy score to beat. <br>
But the NB model did alright.

### LogisticRegression
We've seen the performance of all these fancy models.<br>
But this is a basic binary classification so, let's try the LogisticRegression model which is a rather simpler approach to this type of classification

In [38]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train_scaled, Y_train)
accuracy = round((clf.score(X_test_scaled, Y_test))*100)
print(f'Accuracy of the model: {accuracy}%')

Accuracy of the model: 88%


## Neural Networks
We'll use two deep learning frameworks for this part
### Tensorflow and Pytorch


# Tensorflow(Keras)

In [14]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.optimizers import RMSprop

In [15]:
model = Sequential()
model.add(Dense(29, activation='relu', input_shape=(11,)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 29)                348       
_________________________________________________________________
dropout (Dropout)            (None, 29)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 30        
Total params: 378
Trainable params: 378
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='categorical_crossentropy', 
                         optimizer=RMSprop(), 
                         metrics=['accuracy'])

In [18]:
history = model.fit(X_train_scaled, Y_train,
               batch_size=4,
               epochs=30,
               verbose=2,
               validation_data=(X_test_scaled, Y_test))

Epoch 1/30
184/184 - 0s - loss: 6.4639e-08 - accuracy: 0.4060 - val_loss: 7.1266e-08 - val_accuracy: 0.4022
Epoch 2/30
184/184 - 0s - loss: 6.4639e-08 - accuracy: 0.3774 - val_loss: 7.1266e-08 - val_accuracy: 0.4022
Epoch 3/30
184/184 - 0s - loss: 6.4639e-08 - accuracy: 0.3801 - val_loss: 7.1266e-08 - val_accuracy: 0.4022
Epoch 4/30
184/184 - 0s - loss: 6.4639e-08 - accuracy: 0.3856 - val_loss: 7.1266e-08 - val_accuracy: 0.4022
Epoch 5/30
184/184 - 0s - loss: 6.4639e-08 - accuracy: 0.3638 - val_loss: 7.1266e-08 - val_accuracy: 0.4022
Epoch 6/30
184/184 - 0s - loss: 6.4639e-08 - accuracy: 0.3924 - val_loss: 7.1266e-08 - val_accuracy: 0.4022
Epoch 7/30
184/184 - 0s - loss: 6.4639e-08 - accuracy: 0.3638 - val_loss: 7.1266e-08 - val_accuracy: 0.4022
Epoch 8/30
184/184 - 0s - loss: 6.4639e-08 - accuracy: 0.3787 - val_loss: 7.1266e-08 - val_accuracy: 0.4022
Epoch 9/30
184/184 - 0s - loss: 6.4639e-08 - accuracy: 0.3842 - val_loss: 7.1266e-08 - val_accuracy: 0.4022
Epoch 10/30
184/184 - 0s - l

# Pytorch

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [31]:
X_train_tensor = torch.from_numpy(X_train_scaled.astype(np.float32))
Y_train_tensor = torch.from_numpy(Y_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test_scaled.astype(np.float32))
Y_test_tensor = torch.from_numpy(Y_test.astype(np.float32))
input_features = X_train_tensor.shape[1]
learning_rate = 0.01

In [33]:
# Build the model --- fx = wx + b, sigmoid at the end.
class Model(nn.Module):
    def __init__(self, input_features):
        super(Model, self).__init__()
        self.l1 = nn.Linear(input_features, 29)
        self.l_relu = nn.LeakyReLU()
        self.l2 = nn.Linear(29, 1)
    def forward(self, x):
        out = self.l1(x)
        out = self.l_relu(out)
        out = self.l2(out)
        out = torch.sigmoid(out)
        return out

model = Model(input_features)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [36]:
# training loop
epochs = 25
for epoch in range(epochs):
    # forward pass and computing loss
    pred = model(X_train_tensor)
    loss = criterion(pred, Y_train_tensor)
    
    # empty gradients
    optimizer.zero_grad()
    # calculate gradients and backward pass
    loss.backward()
    # update model parameters(weights and biases)
    optimizer.step()
    
    # Training info
    if (epoch%2)==0:
        print(f'Epoch: {epoch+1}/{epochs}, Loss: {loss:.2f}')

ValueError: Using a target size (torch.Size([734])) that is different to the input size (torch.Size([734, 1])) is deprecated. Please ensure they have the same size.